In [1]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

# Hackathon #2 - Data Wrangling (Instructor solution)

## Get the Data

### -> Data in files

What files do we have?

In [3]:
!ls ../data

data_0.tsv            data_3.json           test.csv
data_1.csv            requirements.txt
data_2.csv            sample_submission.csv


Our data is spreadout throughout 4 files, all named data_<slice>*. We'll be taking a peek at them to have an idea of what we're dealing with.

In [4]:
! head -1 ../data/data_*

==> ../data/data_0.tsv <==
���xc�data_0.tsv t�ͮ�Yv]�N>��	�����O��a�jRS0�M@J����n�1g�,P	�*w̛�3G�1�����/�������_��?��_����?������?���������?��?���������������o����?������������_���������O���k?w��������o���u����������_���[����������>�۞��p;܂۝��pn��ȹ��o����7���7�$�����8�~e%~?�Ȓ��Y�Iy�4%��!�<4��#5��=����Q��L�~~?�L�޿zGyh�����<4���C$

==> ../data/data_1.csv <==
,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price

==> ../data/data_2.csv <==
,timestamp,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square

==> ../data/data_3.json <==
BZh91AY&SY�'>���[�h � ""Le�<�                                                                                                                                     �                                                                                                                       �                        0  	BB ��(  ��!!HB   


In [30]:
! file ../data/data_0.tsv

../data/data_0.tsv: gzip compressed data, was "data_0.tsv", last modified: Sat Nov 19 13:06:44 2022, max compression, original size modulo 2^32 3068516


In [31]:
! file ../data/data_3.json

../data/data_3.json: bzip2 compressed data, block size = 900k


#### - data_slice: tsv file, compressed 

In [5]:
df_0 = pd.read_csv('../data/data_0.tsv', sep="\t", compression='gzip')
df_0 = df_0.drop("Unnamed: 0", axis =1)

In [6]:
df_0

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,1332817560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1332817620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1332817680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1332817740,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### - data_1: csv with repeated index and Timestamp in different format

In [7]:
df_1 = pd.read_csv('../data/data_1.csv')
df_1 = df_1.drop("Unnamed: 0", axis =1)

In [8]:
df_1

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2012-03-27 03:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-03-27 03:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-03-27 03:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-03-27 03:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-03-27 03:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,2012-06-21 22:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,2012-06-21 22:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,2012-06-21 22:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,2012-06-21 22:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_1["Timestamp"] = pd.to_datetime(df_1["Timestamp"]).astype(int)/10**9

In [10]:
df_1

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1.332818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.332818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.332818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.332818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.332818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,1.340318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1.340318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1.340318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1.340318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_1.to_csv('../data/data_1.csv')

#### - data_2: CSV file with columns in lower case, and with one extra column

In [12]:
df_2 = pd.read_csv('../data/data_2.csv')
df_2 = df_2.drop("Unnamed: 0", axis =1)

In [13]:
df_2

,timestamp,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
0,1340317800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1340317860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1340317920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1340317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1340318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
124994,1347817440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1347817500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1347817560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1347817620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_2.to_csv("../data/data_2.csv")

#### slice-3: JSON file with Nones, values converted to strings, multiple lines

In [17]:
df_3 = pd.read_json('../data/data_3.json', orient='columns', compression='bz2')
df_3["Timestamp"] = pd.to_datetime(df_3["Timestamp"]).astype(int)/10**9

In [18]:
df_3

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1.347818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.347818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.347818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.347818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.347818e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,1.355317e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1.355317e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1.355318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1.355318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge all data together

The _id_ column is present in all files, so we can merge them using this column. We have the following problems to address:
- Missing data on slice-1, slice-2, slice-3
- Column names are lowercased on slice-4, so to merge we have to lowercase them all
- slice-4 has extra columns that should be dropped
- slice-3 has numerical values as string 
- slice-5 and 6 are the same, although slice-5 has column with a duplicated index

As a refresher, this is the possible joins we have in [pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html). We want to ensure that we're not missing any entries as not all files have the same information. -> We need an outer join!

![join_diagram](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg)

#### Fix issues on particular dataframes

In [19]:
# change all column names to lower
df_0.columns= df_0.columns.str.lower()
df_1.columns= df_1.columns.str.lower()
df_2.columns= df_2.columns.str.lower()
df_3.columns= df_3.columns.str.lower()

In [20]:
# set the index
df_0 = df_0.set_index('timestamp') 
df_1 = df_1.set_index('timestamp') 
df_2 = df_2.set_index('timestamp') 
df_3 = df_3.set_index('timestamp') 

In [21]:
# concatenate dataframes - by default, pd.concat uses an outer join on the index. pd.merge could be an alternative
file_data_df = pd.concat([df_0, df_1, df_2, df_3]).sort_index()
print("Concatenated shape: ", file_data_df.shape)

Concatenated shape:  (499996, 8)


In [22]:
file_data_df

,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
timestamp,,,,,,,,
1.325318e+09,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,NaN
1.325318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.325318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.325318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.325318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1.355317e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.355317e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.355318e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
file_data_df.to_csv("file_data.csv")

#### Merging strategy

Before merging, we need to decide how we are going to merge

* look for ids in different files
* group the same ids together
* if the same id is present in multiple files
    * add columns from multiple files
    * if same column is present in different files, take any non null value as the value of that column

In [26]:
! head -n 1 file_data.csv

timestamp,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
